# Install the necessary packages:

In [ ]:
# Core dependencies
! pip install llama-index
! pip install llama-index-core

# Azure-specific dependencies
! pip install azure-search-documents==11.5.1
! pip install llama-index-vector-stores-azureaisearch
! pip install llama-index-embeddings-azure-openai
! pip install llama-index-llms-azure-openai

# Additional tools and utilities
! pip install "crewai[tools]"
! pip install llama-parse
! pip install llama-index-readers-file
! pip install matplotlib
! pip install python-dotenv

# Initialize Azure Services
Set up Azure OpenAI and Azure AI Search services:

In [18]:
import os
from dotenv import load_dotenv
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from llama_index.core import SimpleDirectoryReader, StorageContext, VectorStoreIndex
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.vector_stores.azureaisearch import AzureAISearchVectorStore, IndexManagement
from llama_parse import LlamaParse
from llama_index.core import SimpleDirectoryReader
import nest_asyncio
nest_asyncio.apply()

load_dotenv()

# Environment Variables
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME = "gpt-4o-mini" # I'm using GPT-4o-mini
# AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME = os.getenv("AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME") # I'm using GPT-4o
AZURE_OPENAI_EMBEDDING_DEPLOYED_MODEL_NAME = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYED_MODEL_NAME") # I'm using text-embedding-3-large
SEARCH_SERVICE_ENDPOINT = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")
SEARCH_SERVICE_API_KEY = os.getenv("AZURE_SEARCH_ADMIN_KEY")
INDEX_NAME = "acc-guidelines-index"
LLAMA_CLOUD_API_KEY=os.getenv("LLAMA_CLOUD_API_KEY")

# Initialize Azure OpenAI and embedding models
llm = AzureOpenAI(
    model=AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME,
    deployment_name=AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME,
    api_key=AZURE_OPENAI_API_KEY,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_version="2024-06-01"
)

embed_model = AzureOpenAIEmbedding(
    model=AZURE_OPENAI_EMBEDDING_DEPLOYED_MODEL_NAME,
    deployment_name=AZURE_OPENAI_EMBEDDING_DEPLOYED_MODEL_NAME,
    api_key=AZURE_OPENAI_API_KEY,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_version="2024-06-01"
)

# Initialize search clients
credential = AzureKeyCredential(SEARCH_SERVICE_API_KEY)
index_client = SearchIndexClient(endpoint=SEARCH_SERVICE_ENDPOINT, credential=credential)
search_client = SearchClient(endpoint=SEARCH_SERVICE_ENDPOINT, index_name=INDEX_NAME, credential=credential)


## Parse the ACC Guidelines PDFs Using LlamaIndex Parser

In [11]:
from llama_parse import LlamaParse

# Set up the parser
parser = LlamaParse(
    result_type="markdown"  # Options are "markdown" or "text"
)

# Use SimpleDirectoryReader to parse all PDFs in the directory
file_extractor = {".pdf": parser}
documents = SimpleDirectoryReader(
    input_dir='data/acc', 
    file_extractor=file_extractor, 
    recursive=True
).load_data()

print(f"Number of documents parsed: {len(documents)}")


Started parsing the file under job_id 7402d71a-bb13-470d-be68-a0706ad4a162
.......Started parsing the file under job_id fd88710f-9cb9-4991-a151-de67e8a1e4cd
................Started parsing the file under job_id e5e3fbe4-c5be-4424-aaf0-dca087d289ed
Started parsing the file under job_id 4db2ad68-a3b2-408b-8c4a-e57f8d0902e5
.Started parsing the file under job_id e00adcc5-0e12-4911-9c4a-eaa8cc2c310d
Started parsing the file under job_id 3599a37f-9d0f-41ab-8140-3e55542bdfbb
......Started parsing the file under job_id 9ca16ed2-ef37-4c7f-baeb-eaace66f9757
.........Started parsing the file under job_id 4b24dee7-adac-408c-84dc-1099077dffb5
Started parsing the file under job_id ab0ef587-4984-4384-801a-c06bdf46f493
.Started parsing the file under job_id 93926b3e-f44d-4c19-9dcb-f36a97bf5c13
..............Started parsing the file under job_id f4de4023-0b5f-4167-8f97-f3fdce8977c5
.............Started parsing the file under job_id d1fc9fb8-d361-45c8-981a-06a6c5cd19ba
.Started parsing the file under j

Note: The SimpleDirectoryReader will recursively read all PDFs in the data/acc directory and parse them using the LlamaIndex parser.

## Prepare Documents for Indexing
Initialize Azure AI Search Index with Splitting Logic
In this step, we'll:

Use the TokenTextSplitter to split the documents into manageable chunks suitable for embedding.
Implement logic to check if an existing index should be used or a new one created.
Use your provided code to initialize the AzureAISearchVectorStore and create the index accordingly.

In [13]:
from llama_index.core.extractors import TitleExtractor, QuestionsAnsweredExtractor
from llama_index.core.node_parser import TokenTextSplitter

from llama_index.core.settings import Settings

Settings.llm = llm
Settings.embed_model = embed_model

# Define whether to use an existing index or create a new one
use_existing_index = False  # Change to True if you already have an index

# Initialize the vector store based on whether we're using an existing index or creating a new one
if use_existing_index:
    vector_store = AzureAISearchVectorStore(
        search_or_index_client=index_client,
        index_client=index_client,
        index_name=INDEX_NAME,
        index_management=IndexManagement.VALIDATE_INDEX,
        id_field_key="id",
        chunk_field_key="chunk",
        embedding_field_key="embedding",
        embedding_dimensionality=1536,
        metadata_string_field_key="metadata",
        doc_id_field_key="doc_id",
        language_analyzer="en.lucene",
        vector_algorithm_type="hnsw",
        compression_type="binary"
    )
else:
    vector_store = AzureAISearchVectorStore(
        search_or_index_client=index_client,
        index_client=index_client,
        index_name=INDEX_NAME,
        index_management=IndexManagement.CREATE_IF_NOT_EXISTS,
        id_field_key="id",
        chunk_field_key="chunk",
        embedding_field_key="embedding",
        embedding_dimensionality=1536,
        metadata_string_field_key="metadata",
        doc_id_field_key="doc_id",
        language_analyzer="en.lucene",
        vector_algorithm_type="hnsw",
        compression_type="binary"
    )

# Load documents and create index based on the use_existing_index flag
if use_existing_index:
    # Use the existing index without re-indexing documents
    storage_context = StorageContext.from_defaults(vector_store=vector_store)
    index = VectorStoreIndex(
        storage_context=storage_context,
    )
else:
    # Configure TokenTextSplitter to split documents into manageable chunks
    text_splitter = TokenTextSplitter(
        separator=" ",  # The separator for splitting text (space in this case)
        chunk_size=1024,  # Maximum number of tokens per chunk
        chunk_overlap=102  # Overlap between chunks (I'll use 10%)
    )

    # Initialize the storage context
    storage_context = StorageContext.from_defaults(vector_store=vector_store)

    # Create the index with document splitting using the TokenTextSplitter
    index = VectorStoreIndex.from_documents(
        documents,  # This is the list of parsed documents from LlamaParse
        transformations=[text_splitter],  # Automatically splits the text for chunking
        storage_context=storage_context,
        show_progress=True
    )

Subtype value aml has no mapping, use base class VectorSearchVectorizer.
Subtype value aml has no mapping, use base class VectorSearchVectorizer.
Subtype value aml has no mapping, use base class VectorSearchVectorizer.
c:\Dev\azure-ai-search-python-playground\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating embeddings: 100%|██████████| 58/58 [00:02<00:00, 19.53it/s]


## Query and Retrieve Information
We can now query the index to retrieve relevant information based on a patient's profile.

**Why This Works:**

- **Specific Conditions**: 
  - Focuses on hypertension and type 2 diabetes, which are common co-occurring conditions that significantly impact cardiovascular risk.

- **Patient Preferences**: 
  - Reflects the patient’s preferences for non-invasive treatments and lifestyle modifications.
  - Aligns with modern guidelines for chronic disease management, emphasizing diet, exercise, and medication as needed.

- **Targeting ACC Guidelines**: 
  - The ACC guidelines typically cover best practices for cardiovascular management, especially for patients with diabetes and hypertension.

In [36]:
from llama_index.core.response.notebook_utils import display_response
from llama_index.core.schema import MetadataMode
from IPython.display import Markdown, display

# Define the patient's profile and preferences
patient_profile = {
    "patient_id": "12345",
    "name": "John Doe",
    "age": 65,
    "gender": "Male",
    "height_cm": 175,
    "weight_kg": 85,
    "bmi": 27.8,
    "blood_type": "O+",
    "allergies": ["Penicillin"],
    "current_medications": ["Metformin", "Lisinopril"],
    "chronic_conditions": {
        "hypertension": True,
        "type_2_diabetes": True,
        "high_cholesterol": False,
        "coronary_artery_disease": False,
    },
    "family_medical_history": {"heart_disease": True, "stroke": False, "cancer": False},
    "lifestyle_factors": {
        "smoking": False,
        "alcohol_use": "Occasional",
        "physical_activity_per_week": "2-3 days",
        "diet": "High in vegetables, moderate in carbs",
        "sleep_hours_per_night": 7,
    },
    "recent_lab_results": {
        "hba1c_percent": 6.8,
        "ldl_cholesterol_mg_dl": 120,
        "hdl_cholesterol_mg_dl": 45,
        "blood_pressure_mm_hg": "140/85",
    },
    "vaccination_status": {
        "influenza_vaccine": True,
        "covid_vaccine": True,
        "pneumonia_vaccine": False,
    },
    "surgical_history": ["Appendectomy"],
    "imaging_history": {"last_chest_xray_date": "2023-05-12", "last_mri_date": None},
    "mental_health": {
        "anxiety": False,
        "depression": False,
        "cognitive_function_issues": False,
    },
    "preferences": {
        "preferred_treatment_type": [
            "Non-invasive treatments",
            "Lifestyle modifications",
        ],
        "end_of_life_care": "Yes",
        "pain_management": "Non-opioid",
    },
    "recent_visits": [
        {
            "visit_date": "2024-08-12",
            "reason": "Routine check-up",
            "notes": "Patient showing slight increase in blood pressure, advised to adjust diet.",
        },
        {
            "visit_date": "2024-05-10",
            "reason": "Follow-up for diabetes management",
            "notes": "HbA1c levels stable, continue with current medication.",
        },
    ],
}

# Formulate a query based on patient profile
query_text = (
    f"What are the recommended treatment options from the ACC guidelines for a {patient_profile['age']}-year-old "
    f"{patient_profile['gender']} with hypertension and type 2 diabetes, who prefers "
    f"{', '.join(patient_profile['preferences'])}? Include non-invasive treatments and lifestyle modifications."
)

# Create a query engine
query_engine = index.as_query_engine(llm=llm, similarity_top_k=5)

# Perform the query
response = query_engine.query(query_text)

# Display the main response
print("Patient Decision Aid:")
display_response(response)

# Optionally, also show reference information from the retrieved documents
print("\nReference Information:")
for node in response.source_nodes:
    display(Markdown(f"### From Source Document:\n{node.get_content(metadata_mode=MetadataMode.LLM)}\n"))
    display(Markdown(f"**Source Metadata:** {node.metadata}\n"))


Patient Decision Aid:


**`Final Response:`** For a 65-year-old male with hypertension and type 2 diabetes, the recommended treatment options include:

1. **Lifestyle Modifications**:
   - **Physical Activity**: Engage in at least 30 minutes of moderate-intensity aerobic activity most days of the week.
   - **Weight Management**: Aim for a body mass index (BMI) between 18.5 and 24.9 kg/m². Monitor waist circumference, targeting less than 35 inches for women and 40 inches for men.
   - **Diet**: Follow a heart-healthy diet that emphasizes fruits, vegetables, whole grains, and lean proteins while reducing sodium intake.

2. **Non-Invasive Treatments**:
   - **Blood Pressure Monitoring**: Regularly check blood pressure at home and during medical visits to ensure control.
   - **Team-Based Care**: Collaborate with healthcare providers, including primary care physicians and specialists, to manage diabetes and hypertension effectively.

3. **Pharmacological Options**:
   - **Antihypertensive Medications**: Initiate treatment with medications as needed, considering the patient's overall health and preferences. Common classes include ACE inhibitors, ARBs, or calcium channel blockers.
   - **Diabetes Management**: Metformin is recommended as a first-line pharmacotherapy for managing type 2 diabetes.

4. **End-of-Life Care Considerations**:
   - Engage in discussions about treatment goals and preferences, focusing on quality of life and symptom management.

5. **Pain Management**:
   - Address any pain issues with appropriate non-invasive strategies, such as physical therapy or lifestyle adjustments, while considering medications if necessary.

These recommendations aim to optimize health outcomes while respecting the patient's preferences and overall well-being.


Reference Information:


### From Source Document:
file_path: c:\Dev\azure-ai-search-python-playground\data\acc\smith-et-al-2011-aha-accf-secondary-prevention-and-risk-reduction-therapy-for-patients-with-coronary-and-other.pdf

# AHA/ACCF Secondary Prevention: 2011 Update

# November 29, 2011:2432–46

# Table 1. Continued

|Area for Intervention|Goals|
|---|---|
|Physical activity|At least 30 minutes, 7 days per week (minimum 5 days per week)|
|Weight management|Body mass index: 18.5 to 24.9 kg/m2 Waist circumference: women 35 inches (89 cm), men 40 inches (102 cm)|
|Type 2 diabetes mellitus management| |
|Antiplatelet agents/anticoagulants| |

# Recommendations

# Class I

1. For all patients, the clinician should encourage 30 to 60 minutes of moderate-intensity aerobic activity, such as brisk walking, at least 5 days and preferably 7 days per week, supplemented by an increase in daily lifestyle activities (eg, walking breaks at work, gardening, household work) to improve cardiorespiratory fitness and move patients out of the least fit, least active high-risk cohort (bottom 20%) (54,55,58). (Level of Evidence: B)
2. For all patients, risk assessment with a physical activity history and/or an exercise test is recommended to guide prognosis and prescription (47–52,58). (Level of Evidence: B)
3. The clinician should counsel patients to report and be evaluated for symptoms related to exercise. (Level of Evidence: C)

# Class IIa

1. It is reasonable for the clinician to recommend complementary resistance training at least 2 days per week (59). (Level of Evidence: C)

# Class I

1. Body mass index and/or waist circumference should be assessed at every visit, and the clinician should consistently encourage weight maintenance/reduction through an appropriate balance of lifestyle physical activity, structured exercise, caloric intake, and formal behavioral programs when indicated to maintain/achieve a body mass index between 18.5 and 24.9 kg/m2 (60 – 62,65–70). (Level of Evidence: B)
2. If waist circumference (measured horizontally at the iliac crest) is 35 inches (89 cm) in women and 40 inches (102 cm) in men, therapeutic lifestyle interventions should be intensified and focused on weight management (66 –70). (Level of Evidence: B)
3. The initial goal of weight loss therapy should be to reduce body weight by approximately 5% to 10% from baseline. With success, further weight loss can be attempted if indicated. (Level of Evidence: C)

# Note:

Recommendations below are for prevention of cardiovascular complications.

# Class I

1. Care for diabetes should be coordinated with the patient’s primary care physician and/or endocrinologist. (Level of Evidence: C)
2. Lifestyle modifications including daily physical activity, weight management, blood pressure control, and lipid management are recommended for all patients with diabetes (19,22–24,29,56,58,59,62,66,74,162). (Level of Evidence: B)

# Class IIa

1. Metformin is an effective first-line pharmacotherapy and can be useful if not contraindicated (74 –76). (Level of Evidence: A)
2. It is reasonable to individualize the intensity of blood sugar–lowering interventions based on the individual patient’s risk of hypoglycemia during treatment. (Level of Evidence: C)

# Class IIb

1. Initiation of pharmacotherapy interventions to achieve target HbA1c may be reasonable (71,72,74 – 80). (Level of Evidence: A)
2. A target HbA1c of 7% may be considered. (Level of Evidence: C)
3. Less stringent HbA1c goals may be considered for patients with a history of severe hypoglycemia, limited life expectancy, advanced microvascular or macrovascular complications, or extensive comorbidities, or those in whom the goal is difficult to attain despite intensive therapeutic interventions. (Level of Evidence: C)

# Class I

1. Aspirin 75–162 mg daily is recommended in all patients with coronary artery disease unless contraindicated (64,81,82,116). (Level of Evidence: A)
2. Clopidogrel 75 mg daily is recommended as an alternative for patients who are intolerant of or allergic to aspirin (117). (Level of Evidence: B)
3. A P2Y12 receptor antagonist in combination with aspirin is indicated in patients after ACS or PCI with stent placement (83– 85). (Level of Evidence:


**Source Metadata:** {'file_path': 'c:\\Dev\\azure-ai-search-python-playground\\data\\acc\\smith-et-al-2011-aha-accf-secondary-prevention-and-risk-reduction-therapy-for-patients-with-coronary-and-other.pdf', 'file_name': 'smith-et-al-2011-aha-accf-secondary-prevention-and-risk-reduction-therapy-for-patients-with-coronary-and-other.pdf', 'file_type': 'application/pdf', 'file_size': 722398, 'creation_date': '2024-09-14', 'last_modified_date': '2024-09-14'}


...

**Source Metadata:** {'file_path': 'c:\\Dev\\azure-ai-search-python-playground\\data\\acc\\whelton-et-al-2017-2017-acc-aha-aapa-abc-acpm-ags-apha-ash-aspc-nma-pcna-guideline-for-the-prevention-detection.pdf', 'file_name': 'whelton-et-al-2017-2017-acc-aha-aapa-abc-acpm-ags-apha-ash-aspc-nma-pcna-guideline-for-the-prevention-detection.pdf', 'file_type': 'application/pdf', 'file_size': 7375859, 'creation_date': '2024-09-14', 'last_modified_date': '2024-09-14'}


### From Source Document:
file_path: c:\Dev\azure-ai-search-python-playground\data\acc\whelton-et-al-2017-2017-acc-aha-aapa-abc-acpm-ags-apha-ash-aspc-nma-pcna-guideline-for-the-prevention-detection.pdf

# Whelton et al.

# 2017 High Blood Pressure Clinical Practice Guideline

# MAY 15, 2018: e127–248

# FIGURE 4 Blood Pressure (BP) Thresholds and Recommendations for Treatment and Follow-Up

|Normal BP|Elevated BP|Stage 1 Hypertension|Stage 2 Hypertension|
|---|---|---|---|
|(DBP < 120/80)|BP 120-129/<80|BP 130-139/80-89|BP ≥ 140/90|
|Optimal lifestyle habits|Nonpharmacological therapy (Class I)|Reassess in 3-6 months (Class I)|Consider initiation of pharmacological therapy (Class I)|
|Reassess in 3-6 months (Class IIa)|Nonpharmacological therapy|BP-lowering medication (Class I)|Reassess in 1 month (Class I)|
|Colors correspond to Class of Recommendation in Table 1.|Colors correspond to Class of Recommendation in Table 1.|Colors correspond to Class of Recommendation in Table 1.|Colors correspond to Class of Recommendation in Table 1.|

Note: Using the ACC/AHA Pooled Cohort Equations (S8.1.2-56, S8.1.2-57). Note that patients with DM or CKD are automatically placed in the high-risk category. For initiation of RAS inhibitor or diuretic therapy, assess blood tests for electrolytes and renal function 2 to 4 weeks after initiating therapy.

Consider initiation of pharmacological therapy for stage 2 hypertension with 2 antihypertensive agents of different classes. Patients with stage 2 hypertension and BP ≥ 160/100 mm Hg should be promptly treated, carefully monitored, and subject to upward medication dose adjustment as necessary to control BP. Reassessment includes BP measurement, detection of orthostatic hypotension in selected patients (e.g., older or with postural symptoms), identification of white coat hypertension or a white coat effect, documentation of adherence, monitoring of the response to therapy, reinforcement of the importance of adherence, reinforcement of the importance of treatment, and assistance with treatment to achieve BP target. ACC indicates American College of Cardiology; AHA, American Heart Association; ASCVD, atherosclerotic cardiovascular disease; BP, blood pressure; CKD, chronic kidney disease; DM, diabetes mellitus; and RAS, renin-angiotensin system.

This recommendation is consistent with prior guidelines, such as JNC 7. In addition, for those for whom nonpharmacological therapy has been ineffective, antihypertensive drug treatment should be added in patients with an SBP ≥ 140 mm Hg or a DBP ≥ 90 mm Hg, even in adults who are at lower risk than those included in RCTs. The rationale for drug treatment in patients with an SBP ≥ 140 mm Hg or a DBP ≥ 90 mm Hg and an estimated 10-year risk of CVD <10% is based on several lines of evidence. First, the relationship of SBP with risk of CVD is known to be continuous across levels of SBP and similar across groups that differ in level of absolute risk (S8.1.2-10). Second, the relative risk reduction attributable to BP-lowering medication therapy is consistent across the range of absolute risk observed in trials (S8.1.2-3, S8.1.2-11, S8.1.2-58), supporting the contention that the relative risk reduction may be similar at lower levels of absolute risk. This is the case even in a meta-analysis of trials in adults without clinical CVD and an average SBP/DBP of 146/84 mm Hg (S8.1.2-5). Finally, modeling studies support the effectiveness and cost-effectiveness of treatment of younger, lower-risk patients over the course of their life spans (S8.1.2-12, S8.1.2-13). Although the numbers needed to treat with BP-lowering medications to prevent a CVD event in the short term are greater in younger, lower-risk individuals with hypertension than in older, higher-risk adults with hypertension, the estimated gains in life expectancy attributable to long-term use of BP-lowering medications are correspondingly greater in younger, lower-risk individuals than in older adults with a higher risk.


**Source Metadata:** {'file_path': 'c:\\Dev\\azure-ai-search-python-playground\\data\\acc\\whelton-et-al-2017-2017-acc-aha-aapa-abc-acpm-ags-apha-ash-aspc-nma-pcna-guideline-for-the-prevention-detection.pdf', 'file_name': 'whelton-et-al-2017-2017-acc-aha-aapa-abc-acpm-ags-apha-ash-aspc-nma-pcna-guideline-for-the-prevention-detection.pdf', 'file_type': 'application/pdf', 'file_size': 7375859, 'creation_date': '2024-09-14', 'last_modified_date': '2024-09-14'}


### From Source Document:
file_path: c:\Dev\azure-ai-search-python-playground\data\acc\whelton-et-al-2017-2017-acc-aha-aapa-abc-acpm-ags-apha-ash-aspc-nma-pcna-guideline-for-the-prevention-detection.pdf

Persons

# Recommendations for Treatment of Hypertension in Older Persons

References that support recommendations are summarized in Online Data Supplement 54.

|COR|LOE|RECOMMENDATIONS|
|---|---|---|
|I|A|Treatment of hypertension with a SBP treatment goal of less than 130 mm Hg is recommended for noninstitutionalized ambulatory community-dwelling adults (≥65 years of age) with an average SBP of 130 mm Hg or higher (S10.3.1-1).|
|IIa|C-EO|For older adults (≥65 years of age) with hypertension and a high burden of comorbidity and limited life expectancy, clinical judgment, patient preference, and a team-based approach to assess risk/benefit is reasonable for decisions regarding intensity of BP lowering and choice of antihypertensive drugs.|

# Synopsis

Because of its extremely high prevalence in older adults, hypertension is not only a leading cause of preventable morbidity and mortality but, perhaps more importantly, is under-recognized as a major contributor to premature disability and institutionalization (S10.3.1-2—S10.3.1-5). Both SBP and DBP increase linearly up to the fifth or sixth decade of life, after which DBP gradually decreases while SBP continues to rise (S10.3.1-6). Thus, isolated systolic hypertension is the predominant form of hypertension in older persons (S10.3.1-7,S10.3.1-8). RCTs have clearly demonstrated that


**Source Metadata:** {'file_path': 'c:\\Dev\\azure-ai-search-python-playground\\data\\acc\\whelton-et-al-2017-2017-acc-aha-aapa-abc-acpm-ags-apha-ash-aspc-nma-pcna-guideline-for-the-prevention-detection.pdf', 'file_name': 'whelton-et-al-2017-2017-acc-aha-aapa-abc-acpm-ags-apha-ash-aspc-nma-pcna-guideline-for-the-prevention-detection.pdf', 'file_type': 'application/pdf', 'file_size': 7375859, 'creation_date': '2024-09-14', 'last_modified_date': '2024-09-14'}


# Results Interpretation

**Patient Overview:**
- **Name**: John Doe
- **Age**: 65
- **Gender**: Male
- **Conditions**: Hypertension, Type 2 Diabetes
- **Preferences**: Non-invasive treatments, lifestyle modifications

**Interpretation of Recommended Treatments:**

### Lifestyle Modifications:

- **Physical Activity**:
  - The guidelines recommend 30 to 60 minutes of moderate-intensity aerobic activity at least 5 days a week. For John, this translates to activities such as brisk walking or cycling. Given his history of hypertension and type 2 diabetes, regular aerobic exercise will not only help reduce his blood pressure but also improve glycemic control by enhancing insulin sensitivity.
  - **Medical Rationale**: Exercise is essential for patients with both hypertension and type 2 diabetes. Physical activity increases vascular elasticity, lowers systolic and diastolic blood pressure, and supports glucose metabolism.

- **Weight Management**:
  - A target BMI between 18.5 and 24.9 kg/m² is recommended, with a focus on waist circumference as well (≤40 inches for men). Given John's BMI of 27.8, he is currently in the overweight category. He would benefit from gradual weight loss to lower his cardiovascular risk.
  - **Medical Rationale**: Obesity is a major risk factor for both hypertension and type 2 diabetes. Losing even 5-10% of body weight can significantly reduce blood pressure and improve glucose metabolism, reducing the likelihood of complications such as coronary artery disease.

- **Dietary Changes**:
  - Emphasis is placed on a heart-healthy, balanced diet that includes fruits, vegetables, whole grains, and low sodium intake. This diet, coupled with regular physical activity, will help John manage both his blood pressure and glucose levels.
  - **Medical Rationale**: A diet rich in fiber and low in refined sugars and saturated fats helps in weight management, blood pressure control, and glycemic control, which is critical in preventing complications like coronary artery disease or stroke.

### Diabetes Management:

- The guidelines recommend coordinating diabetes care with John's primary care physician or endocrinologist, ensuring comprehensive management.
- Metformin is recognized as a first-line therapy for type 2 diabetes, and John is already on this medication. The recommendation aligns with current best practices, and maintaining his HbA1c levels (currently at 6.8%) close to the target of 7% is appropriate for his condition.
- **Medical Rationale**: Metformin is effective in improving insulin sensitivity, reducing hepatic glucose production, and aiding in weight management. John's current HbA1c level is within an acceptable range for diabetes management, and continued use of Metformin, alongside lifestyle modifications, will help maintain his glycemic control.

### Blood Pressure and Cardiovascular Risk:

- **Blood Pressure Management**:
  - With a current blood pressure reading of 140/85 mmHg, John's hypertension is not optimally controlled. The continued use of Lisinopril, an ACE inhibitor, is essential to managing his blood pressure and providing renal protection, especially given his diabetes.
  - **Medical Rationale**: ACE inhibitors are first-line agents in managing hypertension in diabetic patients because of their kidney-protective effects. Tight control of blood pressure (<140/90 mmHg) will reduce the risk of diabetic nephropathy and other cardiovascular complications.

- **Antiplatelet Therapy**:
  - Though John does not currently have coronary artery disease, the guidelines suggest that for high-risk patients (such as those with diabetes and hypertension), aspirin therapy (75–162 mg/day) could be considered to reduce the risk of cardiovascular events, particularly if there is a family history of heart disease (as in John's case).
  - **Medical Rationale**: Aspirin can reduce the risk of myocardial infarction in patients at elevated cardiovascular risk. Given John's age and family history of heart disease, it may be prudent to evaluate the potential benefits of low-dose aspirin therapy, in consultation with his healthcare provider.

### Pain Management:

- Given John's preference for non-opioid pain management, the recommendation is to select medications that do not interfere with his diabetes or hypertension treatment. For instance, acetaminophen or NSAIDs (with caution) can be considered for pain relief.
- **Medical Rationale**: Care should be taken with NSAIDs, as they can elevate blood pressure and impair kidney function, especially in patients with diabetes. Acetaminophen is a safer choice for occasional pain but should be used judiciously.

### Monitoring and Behavioral Support:

- **Blood Pressure Monitoring**:
  - Regular blood pressure checks are essential to ensure John's levels remain below the target threshold (<140/90 mmHg). Adjustments to his current medication regimen may be needed if these targets are not met.

- **Blood Sugar Monitoring**:
  - Continuing to monitor his HbA1c levels will help ensure his diabetes remains well-controlled. Periodic reviews of fasting blood sugar and HbA1c are recommended to detect any need for therapeutic adjustments.

- **Behavioral Programs**:
  - If John struggles with maintaining his weight or adopting lifestyle changes, enrolling him in a formal behavioral program that offers counseling on diet, exercise, and weight management could provide additional support.
  - **Medical Rationale**: Behavioral interventions, such as structured weight loss programs or support groups, can help patients like John achieve sustainable lifestyle changes, which are crucial for managing chronic conditions.

### Conclusion:

John's treatment plan should prioritize lifestyle modifications, continued pharmacological management for both diabetes and hypertension, and regular monitoring of his cardiovascular risk factors. The integration of behavioral support programs can further help him adhere to these recommendations. His care should remain coordinated between his primary care provider, endocrinologist, and possibly a cardiologist to ensure comprehensive management of both his diabetes and hypertension.

This patient-centric approach aims to minimize his risk of cardiovascular events, enhance his quality of life, and support his preferences for non-invasive treatments and lifestyle modifications.

# Revised Query Engine to Prepare for Agent

In [ ]:
from llama_index.core.response.notebook_utils import display_response
from llama_index.core.schema import MetadataMode
from IPython.display import Markdown, display

# Define the patient's profile and preferences (same as before)
patient_profile = {
    "patient_id": "12345",
    "name": "John Doe",
    "age": 65,
    "gender": "Male",
    "height_cm": 175,
    "weight_kg": 85,
    "bmi": 27.8,
    "blood_type": "O+",
    "allergies": ["Penicillin"],
    "current_medications": ["Metformin", "Lisinopril"],
    "chronic_conditions": {
        "hypertension": True,
        "type_2_diabetes": True,
        "high_cholesterol": False,
        "coronary_artery_disease": False,
    },
    "family_medical_history": {"heart_disease": True, "stroke": False, "cancer": False},
    "lifestyle_factors": {
        "smoking": False,
        "alcohol_use": "Occasional",
        "physical_activity_per_week": "2-3 days",
        "diet": "High in vegetables, moderate in carbs",
        "sleep_hours_per_night": 7,
    },
    "recent_lab_results": {
        "hba1c_percent": 6.8,
        "ldl_cholesterol_mg_dl": 120,
        "hdl_cholesterol_mg_dl": 45,
        "blood_pressure_mm_hg": "140/85",
    },
    "vaccination_status": {
        "influenza_vaccine": True,
        "covid_vaccine": True,
        "pneumonia_vaccine": False,
    },
    "surgical_history": ["Appendectomy"],
    "imaging_history": {"last_chest_xray_date": "2023-05-12", "last_mri_date": None},
    "mental_health": {
        "anxiety": False,
        "depression": False,
        "cognitive_function_issues": False,
    },
    "preferences": {
        "preferred_treatment_type": [
            "Non-invasive treatments",
            "Lifestyle modifications",
        ],
        "end_of_life_care": "Yes",
        "pain_management": "Non-opioid",
    },
    "recent_visits": [
        {
            "visit_date": "2024-08-12",
            "reason": "Routine check-up",
            "notes": "Patient showing slight increase in blood pressure, advised to adjust diet.",
        },
        {
            "visit_date": "2024-05-10",
            "reason": "Follow-up for diabetes management",
            "notes": "HbA1c levels stable, continue with current medication.",
        },
    ],
}

# Formulate a query based on patient profile
query_text = (
    f"What are the recommended treatment options from the ACC guidelines for a {patient_profile['age']}-year-old "
    f"{patient_profile['gender']} with hypertension and type 2 diabetes, who prefers "
    f"{', '.join(patient_profile['preferences'])}? Include non-invasive treatments and lifestyle modifications."
)

# Create a query engine
query_engine = index.as_query_engine(llm=llm, similarity_top_k=5)

# Perform the query
response = query_engine.query(query_text)

# Display the main response
print("Patient Decision Aid:")
display_response(response)

# Optionally, also show reference information from the retrieved documents
print("\nReference Information:")
for node in response.source_nodes:
    display(Markdown(f"### From Source Document:\n{node.get_content(metadata_mode=MetadataMode.LLM)}\n"))
    display(Markdown(f"**Source Metadata:** {node.metadata}\n"))


In [41]:
from crewai import Agent, Task, Crew, Process
from crewai_tools import LlamaIndexTool
from langchain_openai import AzureChatOpenAI

# Define Azure OpenAI configuration for the LLM
azure_llm = AzureChatOpenAI(
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_API_KEY,
    model=AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME,
    api_version="2024-06-01",
)

# Define the query tool for agents to interact with the ACC guidelines
query_tool = LlamaIndexTool.from_query_engine(
    query_engine,
    name="Medical Guidelines Query Tool",
    description="Use this tool to look up medical guidelines and recommendations.",
)

# Define the patient's profile
patient_profile = {
    "patient_id": "12345",
    "name": "John Doe",
    "age": 65,
    "gender": "Male",
    "chronic_conditions": {
        "hypertension": True,
        "type_2_diabetes": True,
    },
    "preferences": {
        "preferred_treatment_type": [
            "Non-invasive treatments",
            "Lifestyle modifications",
        ],
    },
}

# Embed patient profile information into the agent goals
# Agent: Guideline Expert
guideline_expert = Agent(
    role="Guideline Expert",
    goal=(
        f"Retrieve and summarize relevant ACC guidelines for a {patient_profile['age']}-year-old "
        f"{patient_profile['gender']} patient with hypertension and type 2 diabetes. "
        f"Focus on treatments that align with the patient's preferences for "
        f"{', '.join(patient_profile['preferences']['preferred_treatment_type'])}, including non-invasive treatments and lifestyle modifications."
    ),
    backstory=(
        "You are a seasoned medical professional with a deep understanding of ACC guidelines. "
        "Your task is to provide evidence-based recommendations for managing complex conditions "
        "such as hypertension and diabetes, tailored to the specific patient."
    ),
    tools=[query_tool],
    llm=azure_llm,
    verbose=True,
)

# Agent: Patient Educator
patient_educator = Agent(
    role="Patient Educator",
    goal=(
        f"Translate the summarized guidelines for the {patient_profile['age']}-year-old "
        f"{patient_profile['gender']} patient with hypertension and type 2 diabetes. "
        "Ensure that the explanation is clear, patient-friendly, and addresses common patient concerns."
    ),
    backstory=(
        "You are a health educator with expertise in explaining complex medical information. "
        "Your goal is to simplify the guidelines and make them understandable for the patient."
    ),
    llm=azure_llm,
    verbose=True,
)

# Agent: Treatment Planner
treatment_planner = Agent(
    role="Treatment Planner",
    goal=(
        f"Align treatment options with the {patient_profile['age']}-year-old patient's medical history and preferences. "
        "Suggest next steps that consider the patient's chronic conditions (hypertension, type 2 diabetes) and their preference for "
        f"{', '.join(patient_profile['preferences']['preferred_treatment_type'])}."
    ),
    backstory=(
        "As an experienced healthcare practitioner specializing in personalized medicine, your job is to create "
        "treatment plans that reflect each patient's medical history, current conditions, and personal preferences."
    ),
    llm=azure_llm,
    verbose=True,
)

# Agent: Output Generator
output_generator = Agent(
    role="Output Generator",
    goal=(
        f"Compile the information into an accessible decision aid document for the {patient_profile['age']}-year-old "
        f"{patient_profile['gender']} patient. Ensure the document clearly outlines the patient's condition, treatment recommendations, "
        "and actionable next steps."
    ),
    backstory=(
        "You are a skilled document creator who ensures that all of the medical information is presented clearly "
        "and concisely. Your job is to compile treatment recommendations and educational information into a format "
        "that the patient can easily digest."
    ),
    llm=azure_llm,
    verbose=True,
)

# Define the tasks for each agent
task1 = Task(
    description="Retrieve relevant guidelines for managing hypertension and diabetes from the ACC.",
    expected_output="Summarized guidelines tailored to the patient's condition, including non-invasive treatment options.",
    agent=guideline_expert,
)

task2 = Task(
    description="Translate the retrieved guidelines into patient-friendly language.",
    expected_output="Simplified, patient-friendly explanations of the treatment options.",
    agent=patient_educator,
)

task3 = Task(
    description="Personalize the treatment plan according to the patient's medical history and preferences.",
    expected_output="A treatment plan that takes into account the patient's individual health profile and preferences.",
    agent=treatment_planner,
)

task4 = Task(
    description="Compile the treatment plan into a comprehensive patient decision aid document.",
    expected_output="A final decision aid that outlines the patient's condition, treatment recommendations, and actionable next steps.",
    agent=output_generator,
)

# Create the Crew and define the process flow
crew = Crew(
    agents=[guideline_expert, patient_educator, treatment_planner, output_generator],
    tasks=[task1, task2, task3, task4],
    process=Process.sequential,  # Ensures the tasks are executed in order
    verbose=True,
)

# Execute the multi-step reasoning process
result = crew.kickoff()

# Display the final patient decision aid
print("Final Patient Decision Aid:")
print(result)


2024-09-14 19:12:57,994 - 7676 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


 [2024-09-14 19:12:58][DEBUG]: == Working Agent: Guideline Expert
 [2024-09-14 19:12:58][INFO]: == Starting Task: Retrieve relevant guidelines for managing hypertension and diabetes from the ACC.


> Entering new CrewAgentExecutor chain...
Action: Medical Guidelines Query Tool  
Action Input: {"query": "ACC guidelines for managing hypertension and diabetes in older adults with a focus on non-invasive treatments and lifestyle modifications"}   

For managing hypertension in older adults, particularly those with diabetes, the guidelines recommend several non-invasive treatments and lifestyle modifications. Key recommendations include:

1. **Nonpharmacological Interventions**: These are essential for reducing blood pressure and include:
   - Weight loss
   - Adopting a heart-healthy dietary pattern
   - Reducing sodium intake
   - Increasing dietary potassium
   - Engaging in structured physical activity
   - Limiting alcohol consumption

2. **Blood Pressure Goals**: For older adults (≥65